In [1]:
# Install a pip package in the current Jupyter kernel
import sys
import os
sys.path.append('../')
#sys.path.append('../wdtools')
sys.path.append('../../WD_models')
sys.path.append('../../corv/src')


### General
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.colors as colors

import astropy
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u
from astropy.table import Table, Column, MaskedColumn, join

from tqdm import tqdm
import pandas as pd
from scipy.optimize import curve_fit
import scipy.stats as stats

### Tools
import data_selector as ds
from bisect import bisect_left
import lmfit
import glob

from astroquery.sdss import SDSS
from astroquery.gaia import Gaia

import corv

#corv.sdss.make_catalogs()

E:\corv\models/
using local paths
star and exposure catalogs not found! check paths and run make_catalogs() if you want to use sdss functionality. otherwise ignore.


In [2]:
c = 2.998e10
k = 1.38e-16
h = 6.626e-27
halpha = 6564.61
hbeta = 4862.68
hgamma = 4341.68
hdelta = 4102.89
speed_light = 299792458 #m/s
radius_sun = 6.957e8
mass_sun = 1.9884e30
newton_G = 6.674e-11
pc_to_m = 3.086775e16

In [3]:
catalog = Table.read('../data/verification/sed_radii.fits')
catalog['corv_rv'] = -9999 * np.ones(len(catalog))
catalog['corv_erv'] = -9999 * np.ones(len(catalog))

print(catalog.keys())  

['dist', 'solution_id', 'designation', 'source_id', 'random_index', 'ref_epoch', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_weight_al', 'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error', 'mean_varpi_factor_al', 'astrometric_matched_observations', 'visibility_periods_used', 'astrometric_sigma5d_max', 'frame_rotator_object_type', 'matched_observations', 'duplicated_source', 'phot_g_n_obs', 'phot_g_mean_flux', 'phot_g_mean_f

In [4]:
"""for j in tqdm( range(len(catalog[:200]))):
    p,m,f = np.array(catalog['col_p_m_f'][j].split('-')).astype(float)
    
    try:
        xid = SDSS.query_specobj(plate = p, mjd = m, fiberID = f)
    except ValueError:
        print('unknown error')
        
    if xid == None:
        print('object not found in SDSS... skipping...')
        continue
    try:
        sp = SDSS.get_spectra(matches=xid)
    except:
        print('http error')
        continue
    for i in range(len(sp[0:1])):
        wl = np.array(10**sp[i][1].data['loglam'])
        fl = np.array(sp[i][1].data['flux'])
        ivar = np.array(sp[i][1].data['ivar'])
        
        corvmodel = corv.models.make_koester_model(names = ['a','b','g','d'])
        param_res, rv_res, rv_init = corv.fit.fit_corv(wl, fl, ivar, corvmodel, teff = catalog['teff'][j])
        print(catalog['vr'][j])
        
    catalog['corv_rv'][j] = (rv_res.params['RV'].value)
    catalog['corv_erv'][j] = (rv_res.params['RV'].stderr)"""

"for j in tqdm( range(len(catalog[:200]))):\n    p,m,f = np.array(catalog['col_p_m_f'][j].split('-')).astype(float)\n    \n    try:\n        xid = SDSS.query_specobj(plate = p, mjd = m, fiberID = f)\n    except ValueError:\n        print('unknown error')\n        \n    if xid == None:\n        print('object not found in SDSS... skipping...')\n        continue\n    try:\n        sp = SDSS.get_spectra(matches=xid)\n    except:\n        print('http error')\n        continue\n    for i in range(len(sp[0:1])):\n        wl = np.array(10**sp[i][1].data['loglam'])\n        fl = np.array(sp[i][1].data['flux'])\n        ivar = np.array(sp[i][1].data['ivar'])\n        \n        corvmodel = corv.models.make_koester_model(names = ['a','b','g','d'])\n        param_res, rv_res, rv_init = corv.fit.fit_corv(wl, fl, ivar, corvmodel, teff = catalog['teff'][j])\n        print(catalog['vr'][j])\n        \n    catalog['corv_rv'][j] = (rv_res.params['RV'].value)\n    catalog['corv_erv'][j] = (rv_res.params['

In [5]:
#print(catalog['vr'][0])

In [6]:
"""import scipy.stats as stats

def linear(x):
    return x

temp_catalog = catalog#[np.abs(stats.zscore(catalog['vr'])) < 1].copy()

plt.figure(figsize=(10,10))
plt.grid()

plt.errorbar(catalog['vr'][np.abs(catalog['corv_rv']) < 2500], catalog['corv_rv'][np.abs(catalog['corv_rv']) < 2500], 
             xerr = catalog['dvr'][np.abs(catalog['corv_rv']) < 2500], yerr = catalog['corv_erv'][np.abs(catalog['corv_rv']) < 2500], fmt='o', alpha = 0.6)
plt.plot(catalog['vr'][np.abs(catalog['corv_rv']) < 2500], linear(catalog['vr'][np.abs(catalog['corv_rv']) < 2500]))


ymin, ymax = plt.ylim()
plt.xlabel(r'Vedant RV $[km/s]$', fontsize=18)
plt.ylabel(r'My RV $[km/s]$', fontsize=18)
#plt.colorbar()"""

"import scipy.stats as stats\n\ndef linear(x):\n    return x\n\ntemp_catalog = catalog#[np.abs(stats.zscore(catalog['vr'])) < 1].copy()\n\nplt.figure(figsize=(10,10))\nplt.grid()\n\nplt.errorbar(catalog['vr'][np.abs(catalog['corv_rv']) < 2500], catalog['corv_rv'][np.abs(catalog['corv_rv']) < 2500], \n             xerr = catalog['dvr'][np.abs(catalog['corv_rv']) < 2500], yerr = catalog['corv_erv'][np.abs(catalog['corv_rv']) < 2500], fmt='o', alpha = 0.6)\nplt.plot(catalog['vr'][np.abs(catalog['corv_rv']) < 2500], linear(catalog['vr'][np.abs(catalog['corv_rv']) < 2500]))\n\n\nymin, ymax = plt.ylim()\nplt.xlabel(r'Vedant RV $[km/s]$', fontsize=18)\nplt.ylabel(r'My RV $[km/s]$', fontsize=18)\n#plt.colorbar()"

In [7]:
from astropy.io import ascii

falcon = Table.read('../data/verification/falcon2010.fit')

In [8]:
print(falcon)

   _Glon       _Glat      _RAJ2000    _DEJ2000  ... Simbad    _RA       _DE   
    deg         deg         deg         deg     ...           deg       deg   
----------- ----------- ----------- ----------- ... ------ --------- ---------
 68.6811530 -75.8545186   0.7967083 -18.3660000 ... Simbad   0.79671 -18.36600
 68.6811530 -75.8545186   0.7967083 -18.3660000 ... Simbad   0.79671 -18.36600
107.1548732 -44.5646136   1.1805833  16.8713056 ... Simbad   1.18058  16.87131
 78.0385783 -74.9936026   1.8950000 -16.0921667 ... Simbad   1.89500 -16.09217
 78.0385783 -74.9936026   1.8950000 -16.0921667 ... Simbad   1.89500 -16.09217
103.0899741 -61.0846261   3.4132917   0.3230833 ... Simbad   3.41329   0.32308
103.0899741 -61.0846261   3.4132917   0.3230833 ... Simbad   3.41329   0.32308
 53.4914328 -81.3978308   4.0526250 -23.8351111 ... Simbad   4.05262 -23.83511
 53.4914328 -81.3978308   4.0526250 -23.8351111 ... Simbad   4.05262 -23.83511
 42.7935357 -82.5226498   4.6853750 -25.6117222 ... 

In [9]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, join_skycoord
from astropy import table

catalog['wd_pos'] = SkyCoord(catalog['ra'], catalog['dec'], unit='deg')
falcon['wd_pos'] = SkyCoord(falcon['_RA'], falcon['_DE'], unit='deg')

join_func = table.join_skycoord(5 * u.arcsecond)
falcon_xmatch = table.join(catalog, falcon, join_funcs={'wd_pos': join_skycoord(5 * u.arcsec)})

In [10]:
print(falcon_xmatch.keys())

['wd_pos_id', 'dist', 'solution_id', 'designation', 'source_id', 'random_index', 'ref_epoch', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_weight_al', 'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error', 'mean_varpi_factor_al', 'astrometric_matched_observations', 'visibility_periods_used', 'astrometric_sigma5d_max', 'frame_rotator_object_type', 'matched_observations', 'duplicated_source', 'phot_g_n_obs', 'phot_g_mean_flux', '

In [11]:
for j in tqdm( range(len(falcon_xmatch))):
    p,m,f = np.array(falcon_xmatch['col_p_m_f'][j].split('-')).astype(float)
    
    try:
        xid = SDSS.query_specobj(plate = p, mjd = m, fiberID = f)
    except ValueError:
        print('unknown error')
        
    if xid == None:
        print('object not found in SDSS... skipping...')
        continue
    try:
        sp = SDSS.get_spectra(matches=xid)
    except:
        print('http error')
        continue
    for i in range(len(sp[0:1])):
        wl = np.array(10**sp[i][1].data['loglam'])
        fl = np.array(sp[i][1].data['flux'])
        ivar = np.array(sp[i][1].data['ivar'])
        
        corvmodel = corv.models.make_koester_model(names = ['a','b','g','d'])
        param_res, rv_res, rv_init = corv.fit.fit_corv(wl, fl, ivar, corvmodel)
        print(str(rv_res.params['RV'].value) + '+/-' + str(rv_res.params['RV'].stderr))
        
    falcon_xmatch['corv_rv'][j] = (rv_res.params['RV'].value)
    falcon_xmatch['corv_erv'][j] = (rv_res.params['RV'].stderr)

  0%|                                                                                           | 0/97 [00:00<?, ?it/s]C:\Users\arsen\anaconda3\lib\site-packages\astroquery\sdss\core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
C:\Users\arsen\anaconda3\lib\site-packages\astropy\table\table.py:3486: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other

  4%|███▎                                                                            | 41/1000 [00:37<14:41,  1.09it/s]Traceback (most recent call last):
  File "C:\Users\arsen\anaconda3\lib\site-packages\emcee\ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "C:\Users\arsen\anacon

emcee: Exception while calling your likelihood function:
  params: [83.51973492]
  args: (<function fit_rv.<locals>.<lambda> at 0x000001B9D85455E0>, Parameters([('teff', <Parameter 'teff', value=32963.500409208566 (fixed), bounds=[3001:39999]>), ('logg', <Parameter 'logg', value=8.182728870238082 (fixed), bounds=[4.51:9.49]>), ('RV', <Parameter 'RV', value=83.51973492061717, bounds=[-2500:2500]>), ('res', <Parameter 'res', value=1 (fixed), bounds=[0:inf]>)]), ['RV'], array([[-2500,  2500]]))
  kwargs: {'is_weighted': True, 'float_behavior': 'posterior', 'userargs': [], 'userkws': {}, 'nan_policy': 'raise'}
  exception:


KeyboardInterrupt: 

In [ ]:
print(falcon_xmatch['corv_erv'])

In [ ]:
import scipy.stats as stats

def linear(x):
    return x

temp_catalog = catalog#[np.abs(stats.zscore(catalog['vr'])) < 1].copy()

plt.figure(figsize=(10,10))
plt.style.use('../stefan.mplstyle')

plt.errorbar(falcon_xmatch['Adp-V'][np.abs(falcon_xmatch['corv_rv']) < 2500], falcon_xmatch['corv_rv'][np.abs(falcon_xmatch['corv_rv']) < 2500], 
             xerr = falcon_xmatch['e_Adp-V'][np.abs(falcon_xmatch['corv_rv']) < 2500], yerr = falcon_xmatch['corv_erv'][np.abs(falcon_xmatch['corv_rv']) < 2500], fmt='o', alpha = 0.6)
plt.plot(falcon_xmatch['vr'][np.abs(falcon_xmatch['corv_rv']) < 2500], linear(falcon_xmatch['vr'][np.abs(falcon_xmatch['corv_rv']) < 2500]))


ymin, ymax = plt.ylim()
plt.xlabel(r'Falcon RV $[km/s]$', fontsize=18)
plt.ylabel(r'My RV $[km/s]$', fontsize=18)
#plt.colorbar()

## Sirius Validation
Sirius B
* Right ascension: 06h 45m 09.0s
* Declination: −16° 43′ 06″RightRight

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table

siriusa = SkyCoord('06 45 08.917 −16 42 58.02',  unit=(u.hourangle, u.deg), frame='icrs') 
siriusb = SkyCoord('06 45 09 -16 43 06',  unit=(u.hourangle, u.deg), frame='icrs')

sirius = Table([['sirius_a', 'sirius_b'], [siriusa.ra.deg, siriusb.ra.deg], [siriusa.dec.deg, siriusb.dec.deg],[siriusa, siriusb]], names=('star', 'ra', 'dec', 'skycoord'))
print(sirius)

radius = u.Quantity(30.0, u.arcmin)

In [ ]:
print(siriusa.ra, siriusa.dec)
print(siriusb.ra, siriusb.dec)

result = Gaia.query_object(siriusa, radius = radius)
print(len(result))

In [ ]:
result['skycoord'] = SkyCoord(result['ra'], result['dec'], unit=(u.deg, u.deg))
#catalog = join(result, sirius, keys='ra', join_type='inner')

In [ ]:
plt.scatter(result['ra'], result['dec'], s=15, c=result['parallax'])
plt.vlines(sirius['ra'], ymin=-16.74, ymax=-16.69, alpha=0.8) 
plt.hlines(sirius['dec'], xmin=101.26, xmax=101.31, alpha=0.8) 
plt.colorbar()

In [ ]:
from astroquery.sdss import SDSS
from astroquery.gaia import Gaia

j1 = Gaia.cone_search_async(siriusa, radius)
#j2 = Gaia.cone_search_async(siriusb, radius)

In [ ]:
sirius_a_gaia = j1.get_results()[0]
#sirius_b_gaia = j2.get_results()[0]

In [ ]:
#print(sirius_a_gaia['solution_id'] == sirius_b_gaia['solution_id'])
print(sirius_a_gaia['phot_g_mean_mag'])